<a href="https://colab.research.google.com/github/ValeriaYashan/analisis_datos/blob/main/Explorador_de_pel%C3%ADculas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install dash
!pip install plotly
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import sqlite3

In [46]:
import csv

# Datos de películas
movies_data = [
    {'Title': 'The Shawshank Redemption', 'Year': 1994, 'Genre': 'Drama', 'Director': 'Frank Darabont'},
    {'Title': 'The Godfather', 'Year': 1972, 'Genre': 'Crime', 'Director': 'Francis Ford Coppola'},
    {'Title': 'The Dark Knight', 'Year': 2008, 'Genre': 'Action', 'Director': 'Christopher Nolan'},
    {'Title': 'Schindler\'s List', 'Year': 1993, 'Genre': 'Biography', 'Director': 'Steven Spielberg'},
    {'Title': 'Pulp Fiction', 'Year': 1994, 'Genre': 'Crime', 'Director': 'Quentin Tarantino'}
]

# Escribir datos en un archivo CSV
with open('movies.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Title', 'Year', 'Genre', 'Director']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for movie in movies_data:
        writer.writerow(movie)

In [47]:
# Datos de películas a agregar
new_movies_data = [
    {'Title': 'Forrest Gump', 'Year': 1994, 'Genre': 'Drama', 'Director': 'Robert Zemeckis'},
    {'Title': 'The Lord of the Rings: The Return of the King', 'Year': 2003, 'Genre': 'Adventure', 'Director': 'Peter Jackson'},
    {'Title': 'The Matrix', 'Year': 1999, 'Genre': 'Action', 'Director': 'Lana Wachowski, Lilly Wachowski'},
    {'Title': 'Inception', 'Year': 2010, 'Genre': 'Action', 'Director': 'Christopher Nolan'},
    {'Title': 'Fight Club', 'Year': 1999, 'Genre': 'Drama', 'Director': 'David Fincher'},
    {'Title': 'The Lord of the Rings: The Fellowship of the Ring', 'Year': 2001, 'Genre': 'Adventure', 'Director': 'Peter Jackson'},
    {'Title': 'The Silence of the Lambs', 'Year': 1991, 'Genre': 'Crime', 'Director': 'Jonathan Demme'},
    {'Title': 'The Shawshank Redemption', 'Year': 1994, 'Genre': 'Drama', 'Director': 'Frank Darabont'},
    {'Title': 'The Godfather: Part II', 'Year': 1974, 'Genre': 'Crime', 'Director': 'Francis Ford Coppola'},
    {'Title': 'The Green Mile', 'Year': 1999, 'Genre': 'Crime', 'Director': 'Frank Darabont'},
    {'Title': 'Goodfellas', 'Year': 1990, 'Genre': 'Biography', 'Director': 'Martin Scorsese'},
    {'Title': 'The Lord of the Rings: The Two Towers', 'Year': 2002, 'Genre': 'Adventure', 'Director': 'Peter Jackson'},
    {'Title': 'Se7en', 'Year': 1995, 'Genre': 'Crime', 'Director': 'David Fincher'},
    {'Title': 'The Silence of the Lambs', 'Year': 1991, 'Genre': 'Crime', 'Director': 'Jonathan Demme'},
    {'Title': 'The Pianist', 'Year': 2002, 'Genre': 'Biography', 'Director': 'Roman Polanski'},
    {'Title': 'The Departed', 'Year': 2006, 'Genre': 'Crime', 'Director': 'Martin Scorsese'},
    {'Title': 'Gladiator', 'Year': 2000, 'Genre': 'Action', 'Director': 'Ridley Scott'},
    {'Title': 'The Prestige', 'Year': 2006, 'Genre': 'Drama', 'Director': 'Christopher Nolan'},
    {'Title': 'American History X', 'Year': 1998, 'Genre': 'Drama', 'Director': 'Tony Kaye'},
    {'Title': 'The Lion King', 'Year': 1994, 'Genre': 'Animation', 'Director': 'Roger Allers, Rob Minkoff'},
    {'Title': 'Saving Private Ryan', 'Year': 1998, 'Genre': 'Drama', 'Director': 'Steven Spielberg'},
    {'Title': 'The Sixth Sense', 'Year': 1999, 'Genre': 'Drama', 'Director': 'M. Night Shyamalan'},
    {'Title': 'The Intouchables', 'Year': 2011, 'Genre': 'Biography', 'Director': 'Olivier Nakache, Éric Toledano'},
    {'Title': 'The Departed', 'Year': 2006, 'Genre': 'Crime', 'Director': 'Martin Scorsese'},
    {'Title': 'The Green Mile', 'Year': 1999, 'Genre': 'Crime', 'Director': 'Frank Darabont'},
    {'Title': 'The Lives of Others', 'Year': 2006, 'Genre': 'Drama', 'Director': 'Florian Henckel von Donnersmarck'}
]

# Abrir el archivo CSV original en modo de escritura (append) y agregar nuevos datos
with open('movies.csv', 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Title', 'Year', 'Genre', 'Director']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    # Escribir los nuevos datos
    for movie in new_movies_data:
        writer.writerow(movie)

In [48]:
# Cargar los datos de películas desde un archivo CSV
df = pd.read_csv('movies.csv')

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Estilos CSS personalizados
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Definir el diseño de la aplicación
app.layout = html.Div([
    html.H1("Explorador de Películas", style={'textAlign': 'center', 'marginBottom': '30px', 'color': '#333'}),  # Título del proyecto

    # Filtro para seleccionar género
    html.Div([
        html.Label("Género:", style={'fontWeight': 'bold', 'marginRight': '10px'}),
        dcc.Dropdown(id='genre-dropdown',
                     options=[{'label': genre, 'value': genre} for genre in df['Genre'].unique()],
                     value=df['Genre'].unique()[0],  # Valor predeterminado: primer género en la lista
                     style={'width': '50%', 'marginRight': '20px'})
    ], style={'marginBottom': '30px'}),

    # Tabla de datos de películas
    html.Div(id='data-table', style={'maxWidth': '800px', 'margin': 'auto'})
])

# Definir la lógica de la aplicación para actualizar la tabla de datos según el género seleccionado
@app.callback(
    Output('data-table', 'children'),
    [Input('genre-dropdown', 'value')]
)
def update_data_table(selected_genre):
    # Filtrar el DataFrame de películas según el género seleccionado
    filtered_df = df[df['Genre'] == selected_genre]

    # Crear una tabla de datos con Dash HTML Components
    table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in filtered_df.columns]), style={'backgroundColor': '#f2f2f2'}),  # Encabezados de la tabla
        html.Tbody([
            html.Tr([html.Td(filtered_df.iloc[i][col]) for col in filtered_df.columns])  # Filas de datos
            for i in range(min(len(filtered_df), 10))  # Mostrar solo las primeras 10 filas
        ])
    ], style={'borderCollapse': 'collapse', 'border': '1px solid #ddd', 'width': '100%', 'textAlign': 'left'})

    return table

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, external_stylesheets=external_stylesheets)

<IPython.core.display.Javascript object>

GUARDAR EN BASE DE DATOS

In [49]:
# Conexión a la base de datos SQLite
conn = sqlite3.connect('movies.db')

# Crear una tabla de películas si no existe
conn.execute('''CREATE TABLE IF NOT EXISTS movies (
                    Title TEXT,
                    Year INTEGER,
                    Genre TEXT,
                    Director TEXT
                )''')

# Insertar datos de películas en la tabla
for movie in df.to_dict('records'):
    conn.execute('''INSERT INTO movies (Title, Year, Genre, Director)
                    VALUES (?, ?, ?, ?)''', (movie['Title'], movie['Year'], movie['Genre'], movie['Director']))

# Guardar los cambios y cerrar la conexión
conn.commit()
conn.close()

In [51]:
# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Estilos CSS personalizados
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Definir el diseño de la aplicación
app.layout = html.Div([
    html.H1("Explorador de Películas", style={'textAlign': 'center', 'marginBottom': '30px', 'color': '#333'}),  # Título del proyecto

    # Botón para actualizar los datos de la base de datos
    html.Button("Actualizar Datos", id="update-button", n_clicks=0, style={'marginBottom': '20px'}),

    # Botón para borrar los filtros
    html.Button("Borrar Filtros", id="clear-filters-button", n_clicks=0, style={'marginBottom': '20px', 'marginLeft': '20px'}),

    # Tabla de datos de películas
    html.Div(id='data-table', style={'maxWidth': '800px', 'margin': 'auto'})
])

# Definir la función para obtener los datos de la base de datos
def get_data_from_database():
    # Conexión a la base de datos SQLite
    conn = sqlite3.connect('movies.db')
    # Ejecutar una consulta SQL para obtener todos los datos de la tabla de películas
    df = pd.read_sql_query("SELECT * FROM movies", conn)
    # Cerrar la conexión
    conn.close()
    return df

# Definir la lógica de la aplicación para mostrar los datos de la base de datos en una tabla
@app.callback(
    Output('data-table', 'children'),
    [Input('update-button', 'n_clicks'),
     Input('clear-filters-button', 'n_clicks')]
)
def update_data_table(update_clicks, clear_clicks):
    # Obtener los datos de la base de datos
    df = get_data_from_database()

    # Si se hace clic en el botón "Borrar Filtros", restablecer los filtros
    if clear_clicks > 0:
        # Aquí puedes agregar la lógica para restablecer los filtros
        print("Filtros borrados")

    # Crear una tabla de datos con Dash HTML Components
    table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in df.columns])),  # Encabezados de la tabla
        html.Tbody([
            html.Tr([html.Td(df.iloc[i][col]) for col in df.columns])  # Filas de datos
            for i in range(len(df))
        ])
    ], style={'borderCollapse': 'collapse', 'border': '1px solid #ddd', 'width': '100%', 'textAlign': 'left'})

    return table

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, external_stylesheets=external_stylesheets)

<IPython.core.display.Javascript object>